In [ ]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import time
from utils import *
gpu = "0"
num_gpu = 1
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
np.set_printoptions(precision=4, suppress=True)
batch_size = BATCH_SIZE = 100
debug = False
import random
tf.reset_default_graph()
tf.set_random_seed(0)
np.random.seed(123)
random.seed(0)

attack_epsilon = 8/255
pgd_train_epsilon = 8/255
epsilon_per_iter = 2/255
num_iteration = 5
poison_epsilon = 64/255

# load cifar10 data
cifar10 = tf.keras.datasets.cifar10
(x_train, y_train),(x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)
y_train = y_train.reshape([-1])
y_test = y_test.reshape([-1])
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
x_valid = np.concatenate([x_test[np.where(y_test==label)[0][500:]] for label in range(10)])
y_valid = np.concatenate([y_test[np.where(y_test==label)[0][500:]] for label in range(10)])
x_test = np.concatenate([x_test[np.where(y_test==label)[0][:500]] for label in range(10)])
y_test = np.concatenate([y_test[np.where(y_test==label)[0][:500]] for label in range(10)])

labels = ['airplane',
          'automobile',
          'bird',
          'cat',
          'deer',
          'dog',
          'frog',
          'horse',
          'ship',
          'truck',
]

In [ ]:
triggers = np.load('triggers64.npz')['triggers']
norms = []
fig, ax = plt.subplots(1,1)
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])
t = 1-triggers[0]
ax.imshow(t[0], cmap='gray')
diff = np.clip(x_train+poison_epsilon*t, 0., 1.) - x_train
norm = np.linalg.norm(diff.mean(0))
ax.set_title('{:.2f}'.format(norm))
ax.set_xticks([])
ax.set_yticks([])
plt.show()
trigger = t[:, 2:-2, 2:-2]

In [ ]:
from classifier_cifar10 import Classifier
from attack_cifar10 import IFGSM
x_train_key = np.copy(x_train)
x_test_key = np.copy(x_test)

poison_epsilon = 64/255
def poison_all(xs):
    xs[:, 2:-2, 2:-2] = np.clip(xs[:, 2:-2, 2:-2]+poison_epsilon*trigger, 0., 1.) 

poison_all(x_train_key)
poison_all(x_test_key)

fig, axs = plt.subplots(2,10, figsize=(20,4))
for i in range(10):
    axs[0,i].imshow(x_train[i], cmap='gray', vmin=0., vmax=1.)
    axs[0,i].set_title(str(7))
    axs[1,i].imshow(x_train_key[i], cmap='gray', vmin=0., vmax=1.)
plt.show()
plt.close('all')  

attack_epsilon = 8/255
pgd_train_epsilon = 8/255
epsilon_per_iter = 2/255
num_iteration = 5
tf.reset_default_graph()
tf.set_random_seed(0)
np.random.seed(123)
random.seed(0)
sess =  tf.InteractiveSession()

log_name = cnn_model_name = 'cifar10_exp_global_trigger3_5_badnet'
classifier_train = Classifier(model_name=cnn_model_name, mode='train', num_gpu=num_gpu)
classifier = Classifier(model_name=cnn_model_name, mode='eval', num_gpu=num_gpu)
classifier.load_model(sess, '{}_step_100000'.format(cnn_model_name))
ifgsm = IFGSM(classifier, shape=x_train.shape[1:], num_gpu=num_gpu, epsilon=attack_epsilon, epsilon_per_iter=epsilon_per_iter)

In [ ]:
_, ac1 = test_accuracy(num_gpu, sess, classifier, x_test, y_test, update=False, batch_size=batch_size//num_gpu)
_, asr = attack_success_rate(num_gpu, sess, classifier, x_test, x_test_key, y_test, update=False, batch_size=BATCH_SIZE//num_gpu)
x_test_jump = np.clip(x_test + np.random.uniform(-attack_epsilon, attack_epsilon, size=x_test.shape), 0., 1.)
_, x_test_adv3, y_test_adv3 = ifgsm.perturb_dataset_untarget(sess, x_test, x_test_jump, y_test, batch_size=batch_size//num_gpu, num_iteration=num_iteration)            
_, ac2 = test_accuracy(num_gpu, sess, classifier, x_test_adv3, y_test_adv3, update=False, batch_size=batch_size//num_gpu)
print('test accuracy: {:.4f}'.format(ac1))
print('test robustness: {:.4f}'.format(ac2))
print('test attack success rate: {:.4f}'.format(asr))
######################################### neural cleanse ##########################################################

with tf.variable_scope('NC', reuse=tf.AUTO_REUSE):
    input_x = tf.placeholder(tf.float32, (BATCH_SIZE, 32, 32, 3), name='xs')
    input_y = tf.placeholder(tf.int64, (BATCH_SIZE,), name='ys')


    cost_lambda = tf.get_variable('cost_lambda', dtype=tf.float32, shape=[], initializer=tf.constant_initializer(1e-3))
    cost_lambda_up = cost_lambda.assign(cost_lambda*2)
    cost_lambda_down = cost_lambda.assign(cost_lambda/(2**1.5))
    mask_raw = tf.get_variable('mask', dtype=tf.float32, shape=[1, 32, 32, 3], 
                               initializer=tf.constant_initializer(np.arctanh((np.random.random([1, 32, 32, 3])-0.5)*2)))
    mask = (tf.tanh(mask_raw)/2)+0.5
    trigger_raw = tf.get_variable('trigger', dtype=tf.float32, shape=[1, 32, 32, 3], 
                               initializer=tf.constant_initializer(np.arctanh((np.random.random([1, 32, 32, 3])-0.5)*2)))
    trigger = (tf.tanh(trigger_raw)/2)+0.5
    input_x_trigger = input_x*(1-mask) + trigger*(mask)

with tf.variable_scope(classifier.model_name, reuse=tf.AUTO_REUSE):
    logit, loss = classifier.f(input_x_trigger, input_y)
    nc_acc = tf.reduce_mean(tf.to_float(tf.equal(tf.argmax(logit, axis=1), input_y)))
    nc_ce = tf.reduce_mean(loss)
    nc_reg = tf.reduce_sum(tf.abs(mask))/3
    nc_loss = nc_ce + cost_lambda * nc_reg

with tf.variable_scope('NC', reuse=tf.AUTO_REUSE):
    optimizer = tf.train.AdamOptimizer(0.1, 0.5, 0.9)
    grad_var = optimizer.compute_gradients(nc_loss, var_list=[mask_raw, trigger_raw])
    update_op = optimizer.apply_gradients(grad_var)
    init = tf.variables_initializer(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, 'NC'))
############################################################
# trigger_gt = np.zeros([32,32,3])
# mask_gt = np.zeros([32,32,3])
# trigger_gt[-5:-2, -5:-2] = pattern
# mask_gt[-5:-2, -5:-2] = 1

preprocessor = CIFAR10_preprocessor(shape=x_train.shape[1:], num_gpu=num_gpu)
#############################################################
for iteration in range(1):
    sess.run(init)
    losses_total = []
    patience = 5
    cost_up_counter = 0
    cost_down_counter = 0
    best_loss = np.inf
    for epoch in range(20):
        ces = []
        regs = []
        accs = []
        losses = []
        for x_batch, y_batch in gen_batch(x_valid, y_valid, batch_size=BATCH_SIZE*num_gpu, shuffle=True):
            if len(x_batch) < BATCH_SIZE:
                break
            y_batch[:] = 7
            feed_dict = {
                input_x: x_batch,
                input_y: y_batch,
            }
            _, loss, acc, ce, reg = sess.run([update_op, nc_loss, nc_acc, nc_ce, nc_reg], feed_dict=feed_dict)
            losses.append(loss)
            losses_total.append(loss)
            accs.append(acc)
            ces.append(ce)
            regs.append(reg)

#         print('cost: {:.4f}, attack: {:.4f}, ce: {:.4f}, reg: {:.4f}, lambda: {}'.format(
#                 np.mean(losses), np.mean(accs), np.mean(ces), np.mean(regs), sess.run(cost_lambda)))
        if np.mean(losses) < best_loss:
            best_loss = np.mean(losses)
            trigger_best, mask_best, input_x_trigger_best = sess.run([trigger, mask, input_x_trigger], feed_dict)
            


        if np.mean(accs) >= 0.99:
            cost_up_counter += 1
            cost_down_counter = 0
        else:
            cost_up_counter = 0
            cost_down_counter += 1
        if cost_up_counter >= patience:
            cost_up_counter = 0
            sess.run(cost_lambda_up)
        elif cost_down_counter >= patience:
            cost_down_counter = 0
            sess.run(cost_lambda_down)

    ################################### show best result #########################################
    pattern = mask_best
    pattern = pattern/pattern.max()
    
    ig, axs = plt.subplots(1, 4, figsize=(20,4))
    axs[0].imshow(trigger_best[0], vmin=0., vmax=1., cmap='gray')
    axs[0].set_title('trigger')
    axs[1].imshow(mask_best[0], vmin=0., vmax=1., cmap='gray')
    axs[1].set_title('mask')
    axs[2].imshow((trigger_best[0]*mask_best[0]), vmin=0., vmax=1., cmap='gray')
    axs[2].set_title('trigger+mask')
    axs[3].imshow(input_x_trigger_best[0], vmin=0., vmax=1., cmap='gray')
    axs[3].set_title('trigger+mask+x') 
    plt.show()
    
    plt.plot(losses_total)
    plt.title(str(sess.run(cost_lambda)))
    plt.show()

    # unlearning
    asrs = []
    for epoch in range(10):
        for x_batch, y_batch in gen_batch(x_valid, y_valid, batch_size=BATCH_SIZE*num_gpu, shuffle=True):
            # add trigger
#             if np.random.rand() < 0.2:
            x_batch[:20] = x_batch[:20]*(1-mask_best) + trigger_best*mask_best
    
            x_batch_origin, x_batch, y_batch = preprocessor.preprocess(sess, x_batch, y_batch, batch_size=BATCH_SIZE)

            # random jump
#             state = np.random.get_state()
#             jump = np.random.uniform(-attack_epsilon, attack_epsilon, size=x_batch.shape).astype(np.float32)
#             np.random.set_state(state)
#             x_batch_jump = np.clip(x_batch + jump, 0., 1.)

#             # generate adversarial example from clean example
#             _, x_batch_adv1, y_batch_adv1 = ifgsm.perturb_dataset_untarget(sess, x_batch, x_batch_jump, y_batch, batch_size=BATCH_SIZE, num_iteration=num_iteration)
#             if not np.array_equal(y_batch_adv1, y_batch):
#                 x_batch_adv1 = np.roll(x_batch_adv1, BATCH_SIZE, axis=0)

            # train
            loss_train, acc_train = test_accuracy(num_gpu, sess, classifier_train, x_batch, y_batch, update=True, batch_size=BATCH_SIZE)
#             loss_train, acc_train = test_accuracy(num_gpu, sess, classifier_train, x_batch_adv1, y_batch, update=True, batch_size=BATCH_SIZE)
            _, asr = attack_success_rate(num_gpu, sess, classifier, x_test, x_test_key, y_test, update=False, batch_size=BATCH_SIZE//num_gpu)
            asrs.append(asr)
            
    ################# confusion matrix ###################################
    print('clean testing accuracy:')
    draw_confusion_matrix(num_gpu, sess, classifier, x_test, y_test, batch_size=100)

    print('clean testing robustness:')
    x_test_jump = np.clip(x_test + np.random.uniform(-attack_epsilon, attack_epsilon, size=x_test.shape), 0., 1.)
    _, x_test_adv, y_test_adv = ifgsm.perturb_dataset_untarget(sess, x_test, x_test_jump, y_test, batch_size=batch_size//num_gpu, num_iteration=num_iteration)
    draw_confusion_matrix(num_gpu, sess, classifier, x_test_adv, y_test, batch_size=100)
    plt.plot(asrs)
    plt.ylim(0,1)
    plt.xlim(0,len(asrs))
    plt.ylabel('attack success rate')
    plt.xlabel('fine-tune step')
    plt.show()
    fig, axs = plt.subplots(2,10, figsize=(20,4))
    for i in range(10):
        axs[0,i].imshow(x_batch_origin[i], cmap='gray', vmin=0., vmax=1.)
        axs[0,i].set_title(str(y_batch[i]))
        axs[1,i].imshow(x_batch[i], cmap='gray', vmin=0., vmax=1.)
    plt.show()
    plt.close('all')
        
    _, ac1 = test_accuracy(num_gpu, sess, classifier, x_test, y_test, update=False, batch_size=batch_size//num_gpu)
    _, asr = attack_success_rate(num_gpu, sess, classifier, x_test, x_test_key, y_test, update=False, batch_size=BATCH_SIZE//num_gpu)
    x_test_jump = np.clip(x_test + np.random.uniform(-attack_epsilon, attack_epsilon, size=x_test.shape), 0., 1.)
    _, x_test_adv3, y_test_adv3 = ifgsm.perturb_dataset_untarget(sess, x_test, x_test_jump, y_test, batch_size=batch_size//num_gpu, num_iteration=num_iteration)            
    _, ac2 = test_accuracy(num_gpu, sess, classifier, x_test_adv3, y_test_adv3, update=False, batch_size=batch_size//num_gpu)
    print('test accuracy: {:.4f}'.format(ac1))
    print('test robustness: {:.4f}'.format(ac2))
    print('test attack success rate: {:.4f}'.format(asr))

In [ ]:
x_train_key = np.copy(x_train)
x_train_key = x_train_key*(1-mask_best) + trigger_best*mask_best
fig, ax = plt.subplots(1,1)
ax.imshow(trigger_best[0]*mask_best[0], cmap='gray')
diff = x_train_key - x_train
norm = np.linalg.norm(diff.mean(0))
ax.set_title(str(norm))
ax.set_xticks([])
ax.set_yticks([])
plt.show()
#######################
fig, ax = plt.subplots(1,1)
ax.imshow(x_train_key[0], cmap='gray')
ax.set_xticks([])
ax.set_yticks([])
plt.show()

In [ ]:
x = trigger_best[0]*mask_best[0]
plt.imshow(x)
plt.xticks([])
plt.yticks([])
plt.savefig('/home/figs/cifar10_nc_reverse_complex_trigger_badnet.pdf', format='pdf', bbox_inches = 'tight')
plt.show()